>## Partie 1 : Génération de données

# Bibliothèques

- On utilise la bibliothèque ``OpenCV`` pour la manipulation des images.
- La bibliothèque ``numpy`` pour la manipulation des tableaux.
- La bibliothèque ``matplotlib`` pour l'affichage des images.
- La bibliothèque ``pickle`` pour la sauvegarde des données.
- La bibliothèque ``os`` pour la manipulation des fichiers.

In [ ]:
import os
import cv2
import pickle
import numpy as np
from matplotlib import pyplot as plt

# Création des données

Le code ci-dessous permet de créer un dataset d'images qui vont être utilisées pour la reconnaissance faciale. Ce dataset contient des images des visages de gens qui utiliseront le notebook. Pour chaque personne, on prend 30 images. On utilise la webcam pour capturer les images.

Le code se déroule comme suit :
- On demande le prénom de l'utilisateur, on demande également s'il est admis ou non.
- On utilise la caméra pour capturer les images, on détecte le visage à l'aide de la méthode ``detectMultiScale`` de la classe ``cv2.CascadeClassifier``.
- On convertit l'image en niveau de gris.
- On redimensionne l'image pour qu'elle soit de taille 50x50.
- Une fois les 30 images capturées, on les stocke dans un tableau.
- Puis, on sauvegarde le tableau dans un fichier ``.pkl``.

La fonction `sauvegarder_donnees` enregistre dans des fichiers distincts à l'aide de la sérialisation avec `pickle` : 
- le prénom
- le statut d'admission
- des images du visage de la personne 

- Voici une explication détaillée de son fonctionnement, étape par étape :

1. **Vérification de l'existence du fichier `prenoms.pkl`** :
   - Si le fichier `prenoms.pkl` n'existe pas dans le répertoire `data/`, la fonction crée ce fichier et y enregistre une liste contenant le prénom passé en argument, répété un nombre de fois égal à `nb_images_collectees`.
   - Si le fichier existe déjà, il est ouvert en mode lecture (`'rb'`), les prénoms précédemment enregistrés sont chargés dans une liste, puis le prénom passé en argument est ajouté à cette liste autant de fois que spécifié par `nb_images_collectees`. La liste mise à jour est ensuite réenregistrée dans le fichier.

2. **Vérification de l'existence du fichier `admis.pkl`** :
   - Un processus similaire est suivi pour le fichier `admis.pkl`, qui stocke les statuts d'admission. Si le fichier n'existe pas, il est créé et une liste contenant le statut d'admission (`admi`) répété est enregistrée. Si le fichier existe, le statut d'admission est ajouté à la liste existante et sauvegardée de nouveau.

3. **Vérification de l'existence du fichier `visages.pkl`** :
   - Pour le fichier `visages.pkl`, qui stocke les images de visages, la vérification est également faite. Si le fichier n'existe pas, il est créé et les données de visage (`visage`) sont enregistrées directement.
   - Si le fichier existe déjà, les données de visage précédemment enregistrées sont chargées, et les nouvelles données de visage sont ajoutées à cette collection. La mise à jour des données est ensuite sauvegardée dans le fichier.

In [ ]:
# On vérifie si le dossier 'data' existe, sinon on le crée
if not os.path.exists("data"):
    os.mkdir("data")

# On définit le nombre d'images de visage à collecter par personne
nb_images_collectees = 30

# Capture de la webcam
camera = cv2.VideoCapture(0) 

# Classifieur pour la détection de visage
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
# Demande à l'utilisateur de saisir son prénom et s'il est admis
prenom = input("Entrez votre prénom : ")
admis = input("Êtes-vous admis ? (oui/non) : ").lower() == "oui"
    
def photo():
    # Initialisation du compteur et de la liste pour stocker les visages
    count = 0
    donnees_visage = []
    
    while True:
        # Capture une image de la vidéo
        ret, frame = camera.read()
        
        if ret:
            # Détecte les visages dans l'image
            coordonnees_visage = cascade_visage.detectMultiScale(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), 1.3, 4)
    
            for (a, b, w, h) in coordonnees_visage:
                # Enregistre une image de visage tous les 10 frames, jusqu'à atteindre le nombre désiré
                if count % 10 == 0 and len(donnees_visage) < nb_images_collectees:
                    donnees_visage.append(cv2.resize(frame[b:b+h, a:a+w, :], (50, 50)))
                    
                # Dessine un rectangle autour de chaque visage détecté
                cv2.rectangle(frame, (a, b), (a+w, b+h), (255, 0, 0), 2)
                
            count += 1
    
            # Affiche l'image avec les rectangles
            cv2.imshow('Visages', frame)
    
            # Quitte la boucle si l'utilisateur appuie sur 'Esc' ou si le nombre d'images collectées est atteint
            if cv2.waitKey(1) == 27 or len(donnees_visage) >= nb_images_collectees:
                break
        else:
            print('Erreur')
            break
        
    cv2.destroyAllWindows()
    camera.release(
    
    return np.asarray(donnees_visage)

def sauvegarder_donnees(prenom, admi, visage):
    # Sauvegarde le prénom dans 'prenoms.pkl', ajoute si le fichier existe déjà
    if 'prenoms.pkl' not in os.listdir('data/'):
        with open('data/prenoms.pkl', 'wb') as file:
            pickle.dump([prenom]*nb_images_collectees, file)
    else:
        with open('data/prenoms.pkl', 'rb') as file:
            prenoms = pickle.load(file)
    
        with open('data/prenoms.pkl', 'wb') as file:
            pickle.dump(prenoms + [prenom]*nb_images_collectees, file)
    
    # Sauvegarde le statut d'admission dans 'admis.pkl', ajoute si le fichier existe déjà
    if 'admis.pkl' not in os.listdir('data/'):
        with open('data/admis.pkl', 'wb') as file:
            pickle.dump([admi]*nb_images_collectees, file)
    else:
        with open('data/admis.pkl', 'rb') as file:
            admis = pickle.load(file)
    
        with open('data/admis.pkl', 'wb') as file:
            pickle.dump(admis + [admi]*nb_images_collectees, file)
    
    # Sauvegarde les données de visage dans 'visages.pkl', ajoute si le fichier existe déjà
    if 'visages.pkl' not in os.listdir('data/'):
        with open('data/visages.pkl', 'wb') as w:
            pickle.dump(donnees_visage, w)
    else:
        with open('data/visages.pkl', 'rb') as w:
            visages = pickle.load(w)
            
        visages = np.append(visages, visage, axis=0)
        with open('data/visages.pkl', 'wb') as w:
            pickle.dump(visages, w)
    
donnees_visage = photo()
sauvegarder_donnees(prenom, admis, donnees_visage)

# Affichage des données

Le code ci-dessous permet d'afficher les images des visages qui ont été capturées. On utilise la méthode ``imshow`` de la bibliothèque ``matplotlib`` pour afficher les images. On affiche une photo par personne enregistrée. On affiche également le prénom de la personne et son statut d'admission.

In [ ]:
# Définit une fonction pour afficher les données stockées
def afficher_donnees():
    # On ouvre et charge la liste des prénoms depuis le fichier 'prenoms.pkl'
    with open('data/prenoms.pkl', 'rb') as file:
        prenoms = pickle.load(file)

    # On ouvre et charge la liste des images de visages depuis le fichier 'visages.pkl'
    with open('data/visages.pkl', 'rb') as file:
        visages = pickle.load(file)

    # On ouvre et charge la liste des statuts d'admission depuis le fichier 'admis.pkl'
    with open('data/admis.pkl', 'rb') as file:
        admis = pickle.load(file)

    # Boucle sur les indices des visages avec un pas égal au nombre total d'images collectées pour chaque personne
    for i in range(0, len(visages), nb_images_collectees):
        plt.figure()
        # On affiche l'image du visage à l'indice i
        plt.imshow(visages[i]) 
        # On affiche le prénom et le statut d'admission de la personne
        plt.title(prenoms[i] + " | Admis: " + str(admis[i]))
        
afficher_donnees()